In [55]:
import json
import sympy as sp

# Pulley Analytical Solutions

This notebook solves analytically for GT-2 and GT-3 pulley tooth profiles. Specifically, it solves for the locations of the relevant arc segment points and centers given the definition of the GT tooth profile. 

![Tooth Profile Diagram](diagrams/drawing_full.svg)  
*Tooth profile diagram of GT-#N tooth profiles. Specific dimensions are for a GT-2 pulley with 10 teeth*

## Solution Strategy

Solving for all of the points at once doesn't seem to complete in a reasonable amount of time, so the solution is aproached in several steps.

### Step 1

![Step 1 Diagram](diagrams/drawing_step_1.svg)  

Solve for AX, AY, BX, BY, ABX, ABY, BCX, BCY. We know that *P (BCX, BCY)*, *P (ABX, ABY)*, and *P (BX, BY)* are colinear since *arc BC* is tangent to *arc AB*. We use this along with the parameter constraints and natural circular constraints to solve for the above variables. Since there are multiple solutions, we will manually evaluate and choose the correct solution.

### Step 2

![Step 2 Diagram](diagrams/drawing_step_2.svg)  

Now, we need to solve for point CD, which will allow us to trivially solve for C as well as D, since they are along the lines BC -> CD and C -> O (O is the origin), and the distance along each line is defined by constants. 

### Step 3

![Step 3 Diagram](diagrams/drawing_step_3.svg)  

Finally, we need to solve for E, F, EF, G, FG. Firstly, we know that the angle $\alpha$ between lines O -> A and O -> G is:

> $\alpha = 2\frac{P}{PD}$

Next, the angle $\theta$ of $\overrightarrow{OB}$ with respect to $\hat{y}$ can be calculated by the formula:

> $\theta = -\operatorname{asin}{\left(\frac{BX}{\sqrt{BX^2+BY^2}} \right)}$  

Since the angle of $\overrightarrow{OA}$ with respect to $\hat{y}$ is opposite $\overrightarrow{OB}$, then the angle of $\overrightarrow{OG}$ with respect to the y axis ($\gamma$) is: 

> $\gamma = -\theta - \alpha$  

E, F, EF, G, and FG are reflected in polar coordinates over the angle bisector of $\angle BOG$, $\delta$, which is defined as:

> $\delta = \frac{\theta + \gamma}{2}$  

The formula for reflecting a point $P (x, y)$ across an angle $\theta$ from the y axis is:

> $x' = - \sqrt{x^{2} + y^{2}} \sin{\left(2 θ + \operatorname{asin}{\left(\frac{x}{\sqrt{x^{2} + y^{2}}} \right)} \right)}$  
> $y' = \sqrt{x^{2} + y^{2}} \cos{\left(2 θ + \operatorname{asin}{\left(\frac{x}{\sqrt{x^{2} + y^{2}}} \right)} \right)}$

In [56]:
# Variables
AX, AY, BX, BY, ABX, ABY, BCX, BCY, RBXY, CX, CY, CDX, CDY, DX, DY, EX, EY, FX, FY, EFX, EFY, GX, GY, FGX, FGY = sp.symbols(
    'AX, AY, BX, BY, ABX, ABY, BCX, BCY, RBXY, CX, CY, CDX, CDY, DX, DY, EX, EY, FX, FY, EFX, EFY, GX, GY, FGX, FGY', real=True)

In [57]:
# Parameters
RAB, RBC, RCD, b, h, PLD, t, PD, P = sp.symbols(
    'RAB, RBC, RCD, b, h, PLD, t, PD, P') 
params = [RAB, RBC, RCD, b, h, PLD, t, P] # PD is derived from P and t
vals = [0.555, 1, 0.15, 0.4, 0.75, 0.254, 10, 2]
param_associations = list(zip(params, vals))

In [58]:
# Substitutions
L = sp.symbols("L")
L_sub = PD / 2 - PLD - h + RAB
PD_sub = P*t/sp.pi

In [59]:
# Equation set 1. The goal is to solve for BCX, BCY, BX, and BY.
equations_1 = [
    AX + BX,
    AY - BY,
    ABX,
    L - ABY,
    RBXY * sp.sin(2 * b / PD) + BCX,
    RBXY * sp.cos(2 * b / PD) - BCY,
    sp.sqrt(BCX**2 + BCY**2) - RBXY,
    (BX - ABX)**2 + (BY - ABY)**2 - RAB**2,
    (BX - BCX)**2 + (BY - BCY)**2 - RBC**2,
    (BCY - ABY) / (BCX - ABX) - (ABY - BY) / (ABX - BX)
]

In [60]:
res_1 = sp.solve(equations_1, AX, AY, BX, BY, ABX,
                 ABY, BCX, BCY, RBXY, dict=True)
                

In [61]:
res_1_sol = res_1[1] # Hand-picked solution which matches the drawing
# Substitute back L and PD
for key, val in res_1_sol.items():
    res_1_sol[key] = val.subs(L, L_sub).subs(PD, PD_sub)

In [62]:
equations_2 = [
    (CDX - BCX)**2 + (CDY - BCY)**2 - (RBC+RCD)**2,
    CDX**2 + CDY**2 - (PD/2 - PLD - RCD)**2
]

In [63]:
res_2 = sp.solve(equations_2, CDX, CDY, dict=True)

In [69]:
res_2_sol = res_2[0] # Hand-picked solution which matches the drawing
# Substitute back PD
for key, val in res_2_sol.items():
    res_2_sol[key] = val.subs(PD, PD_sub)

### Step 3

Final solution is being prepared. We reflect points across the angle bisector of $\angle BOG$

In [70]:
def reflect_point(x, y, theta):
    R = sp.sqrt(x**2 + y**2)
    theta_new = 2*theta + sp.asin(x/sp.sqrt(x**2 + y**2))
    x_new = -R * sp.sin(theta_new)
    y_new = R * sp.cos(theta_new)
    return x_new, y_new

In [76]:
final_sol = [(k, v) for k, v in res_1_sol.items()] + [(k, v) for k, v in res_2_sol.items()]

final_sol.append((CX, CDX-(RCD)/(RCD+RBC)*(CDX - BCX)))
final_sol.append((CY, CDY-(RCD)/(RCD+RBC)*(CDY - BCY)))

# Temporary variables to make calculations quicker when ported
# CDR = Distance of CD from the origin
# δ = Angle of line with respect to y axis which D, C, CD, BC are reflected over
CDR, δ = sp.symbols('CDR, δ') 

final_sol.append((CDR, sp.sqrt(CDX**2+CDY**2)))
final_sol.append((DX, CDX*(CDR + RCD)/(CDR)))
final_sol.append((DY, CDY*(CDR + RCD)/(CDR)))
final_sol.append((δ, (-sp.asin(BX/(sp.sqrt(BX**2+BY**2))) + (sp.asin(BX/(sp.sqrt(BX**2+BY**2))) - 2*P/PD)).subs(PD, PD_sub)/2))

EX_, EY_ = reflect_point(DX, DY, δ)
final_sol.append((EX, EX_))
final_sol.append((EY, EY_))

FX_, FY_ = reflect_point(CX, CY, δ)
final_sol.append((FX, FX_))
final_sol.append((FY, FY_))

EFX_, EFY_ = reflect_point(CDX, CDY, δ)
final_sol.append((EFX, EFX_))
final_sol.append((EFY, EFY_))

GX_, GY_ = reflect_point(BX, BY, δ)
final_sol.append((GX, GX_))
final_sol.append((GY, GY_))

FGX_, FGY_ = reflect_point(BCX, BCY, δ)
final_sol.append((FGX, FGX_))
final_sol.append((FGY, FGY_))

### Evaluate Solution

The equations ought to be evaluated in order and substituted sequentially to achieve efficient execution. The solutions are thus prepared in a list in order, where each subsequent equation could use solutions from the previous equations.

In [77]:
import copy
final_sol_eval = copy.deepcopy(final_sol)

# Evaluate Solution
for i, el in enumerate(final_sol_eval):
    final_sol_eval[i] = (el[0], sp.N(el[1].subs(param_associations).subs(final_sol_eval[:i])))
final_sol_eval

[(AX, -0.468387108234873),
 (AY, 2.43637664761906),
 (BX, 0.468387108234873),
 (BY, 2.43637664761906),
 (ABX, 0),
 (ABY, 2.73409886183791),
 (BCX, -0.375553627323457),
 (BCY, 2.97281306963500),
 (RBXY, 2.99644090113396),
 (CDX, 0.736474689773907),
 (CDY, 2.67973795644111),
 (CX, 0.591427517978598),
 (CY, 2.71796514511857),
 (CDR, 2.77909886183791),
 (DX, 0.776225417962478),
 (DY, 2.82437501811093),
 (δ, -0.314159265358979),
 (EX, 1.03214642799145),
 (EY, 2.74122124127270),
 (FX, 1.11910491555999),
 (FY, 2.54651236538746),
 (EFX, 0.979289910850347),
 (EFY, 2.60084250856823),
 (GX, 1.05313313199211),
 (GY, 2.24638114720646),
 (FGX, 2.05120494695760),
 (FGY, 2.18431141084900)]

In [78]:
final_sol

[(AX,
  (-2*RAB**2*(P*t/(2*pi) - PLD + RAB - h)*sin(pi*b/(P*t))**2 + RAB**2*(P*t/(2*pi) - PLD + RAB - h) + RAB**2*sqrt(RAB**2 - 2*RAB*RBC + RBC**2 + 4*(P*t/(2*pi) - PLD + RAB - h)**2*sin(pi*b/(P*t))**4 - 4*(P*t/(2*pi) - PLD + RAB - h)**2*sin(pi*b/(P*t))**2) - 2*RAB*RBC*(P*t/(2*pi) - PLD + RAB - h)*sin(pi*b/(P*t))**2 + RAB*RBC*(P*t/(2*pi) - PLD + RAB - h) + RAB*RBC*sqrt(RAB**2 - 2*RAB*RBC + RBC**2 + 4*(P*t/(2*pi) - PLD + RAB - h)**2*sin(pi*b/(P*t))**4 - 4*(P*t/(2*pi) - PLD + RAB - h)**2*sin(pi*b/(P*t))**2) + 256*(P*t/(2*pi) - PLD + RAB - h)**3*sin(pi*b/(P*t))**18 - 1152*(P*t/(2*pi) - PLD + RAB - h)**3*sin(pi*b/(P*t))**16 + 2112*(P*t/(2*pi) - PLD + RAB - h)**3*sin(pi*b/(P*t))**14 + 256*(P*t/(2*pi) - PLD + RAB - h)**3*sin(pi*b/(P*t))**12*cos(pi*b/(P*t))**6 - 2016*(P*t/(2*pi) - PLD + RAB - h)**3*sin(pi*b/(P*t))**12 - 384*(P*t/(2*pi) - PLD + RAB - h)**3*sin(pi*b/(P*t))**10*cos(pi*b/(P*t))**6 + 1056*(P*t/(2*pi) - PLD + RAB - h)**3*sin(pi*b/(P*t))**10 + 192*(P*t/(2*pi) - PLD + RAB - h)**3*sin